In [0]:
! pip3 install keras
! pip3 install ann_visualizer
! pip install graphviz
! pip install h5py

In [0]:
from pandas import read_csv
import numpy as np
import keras
import os, sys
from IPython.display import display
from IPython.display import Image as _Imgdis
from PIL import Image
import numpy as np
from time import time
from time import sleep
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import matplotlib.pyplot as plt
import time
from random import randint
import matplotlib.pyplot as plt
import random
from shutil import rmtree
import os
import gc
from keras.layers import BatchNormalization,Activation,MaxPooling2D,AveragePooling2D,Flatten,Dense,Dropout,Input,Conv2D,UpSampling2D,Reshape,Conv2DTranspose
from keras.models import Sequential,Model,model_from_json,load_model
from keras.layers import LSTM,Dropout,InputLayer


> ## Helper function

In [0]:
from ann_visualizer.visualize import ann_viz;
from keras.models import model_from_json

def visualize_model(model,is_json=False,):
  if is_json:
    m = model_from_json(model)
    ann_viz(m, title="Model Visualization")
  else:
      ann_viz(model, title="Model Visualization")



In [0]:
def plot_dense(x,epoch):
  n=9
  pr=3
  fig_size=(20,20)
  fig = plt.figure(figsize=fig_size)
  for i in range(n):
    ar=x[i]
    p=[]
    p.append(ar)
    p.append(ar)
    p.append(ar)
    a1 = fig.add_subplot(n/pr,n/pr, i+1)
    a1.imshow(p,cmap='gray')
    a1.get_xaxis().set_visible(False)
    a1.get_yaxis().set_visible(False)
#     plt.show()
  plt.close()
  fig.savefig("{}/dense/{}.png".format(path,epoch))
 

In [0]:
# #### USE WITH CAUTION ###### #
# use only when needed to reset the progress and start over again
#this function resets the workin directory
from shutil import rmtree
import os

def reset(path,make_other_dir=True):
  
  if make_other_dir==True:
    acc = path+"/acc" 
    loss=path+"/loss"
    out=path+"/out" 
    dense=path+'/dense'
    folder_list=[acc,out,loss,dense]
  else:
    p='/'.join(path.split('/')[:-1])
    p1=p+'/rnn'
    p2=p+'/aec'
    p3=p+'/dataset'
    folder_list=[p,p1,p2,p3]
    
  print(folder_list)
  conformation=input("do you really want to delete all folders i.e acc,loss,out (Y/N)?")
  if conformation == 'Y' or conformation == 'y':
    for f in folder_list:
      try:
        rmtree(f,ignore_errors=False)
      except:
        pass
      try:
        os.mkdir(f)
      except:
        pass
    print("reset sucessfull :"+str(folder_list))

      
    
# reset()

In [0]:
load_dataset_from_npy = True 
load_saved_model=False  

path = F"ai_dance_models/ai_dance_128_dims/aec" 
img_path = F"dataset/480_cleaned_14k_800x480/"
dataset_path=F"dataset/dataset.npy"
dense_dataset_path=F"dataset/dense_dataset.npy"
model_path=path+"/model_weight.h5"


In [0]:
# make all the folders
# ignore if you downloaded the data already
os.mkdir('ai_dance_models')
reset(path,False)
reset(path,True)
os.mkdir('dataset')

> ## Loading dataset


In [0]:
# //load data driectly from images or .npy file.

# Original Dimensions
image_height = 480
image_width = 800
compression_ratio = 4

image_width = int(image_width / compression_ratio)
image_height = int(image_height / compression_ratio)

channels = 1

dataset=None
if load_dataset_from_npy==True:
  dataset=np.load(dataset_path)
  print("Data loaded \nDataset shape: {}".format(dataset.shape))
else:
  folder = img_path
  onlyfiles = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
  print("Working with {0} images".format(len(onlyfiles)))
  print(onlyfiles[:10])

  train_files = []
  for _file in onlyfiles:
      train_files.append(_file)
  train_files.sort()

  print("Files in train_files: {}".format(len(train_files)))

 

  dataset = np.ndarray(shape=(len(train_files), image_height, image_width,channels,),
                       dtype=np.float32)

  i = 0
  for _file in train_files:
      img = load_img(folder + "/" + _file,color_mode='grayscale')  # this is a PIL image
      img.thumbnail((image_width, image_height))
      x = img_to_array(img)  
      x = x.reshape((image_height, image_width,channels))
      x=x/255
      dataset[i] = x
      i += 1
      if i % 250 == 0:
          print("%d images to array" % i)
  print("All images to array!")
  print('saved array to ',dataset_path)
  np.save(dataset_path,dataset)



In [0]:
# check sample image
n=random.randint(0,len(dataset))
img=np.reshape(dataset[n],(image_height,image_width))
plt.imshow(img,cmap='gray')
plt.title("BnW image(input)")
plt.show()





In [0]:
from keras.layers import BatchNormalization,Activation,MaxPooling2D,AveragePooling2D,Flatten,Dense,Dropout,Input,Conv2D,UpSampling2D,Reshape,Conv2DTranspose
from keras.models import Sequential,Model,model_from_json,load_model

In [0]:
filters=32
encoded_dims=128
def Encoder():
  
  inputshape=dataset[0].shape
  input_img= Input(shape=(inputshape))
  x = Conv2D(filters,(3,3), padding='same')(input_img)
  x=Activation('relu')(x)
  x = Conv2D(filters,(3,3), padding='same')(x)
  x=Activation('relu')(x)
  
  x = MaxPooling2D((2,2), padding='same')(x)
  x = Conv2D(filters,(3,3), padding='same')(x)
  x=Activation('relu')(x)
  x = Conv2D(filters,(3,3), padding='same')(x)
  x=Activation('relu')(x)
  
  x = MaxPooling2D((2,2), padding='same')(x)
  x = Conv2D(filters,(3,3), padding='same')(x)
  x=Activation('relu')(x)
  x = Conv2D(filters,(3,3), padding='same')(x)
  x=Activation('relu')(x)
  
  x = MaxPooling2D((2,2), padding='same',)(x)
  x = Conv2D(filters,(3,3), padding='same')(x)
  x=Activation('relu')(x)
  x = Conv2D(8,(3,3), padding='same')(x)
  x=Activation('relu')(x)

  x=Flatten()(x)
  x=Dense(512)(x)
  x=Activation('relu')(x)


  x=Dense(encoded_dims)(x)
  x=Activation('relu')(x)

  encoder=Model(input_img,x)
  return encoder


def Decoder():
  
  inp=Input(shape=(encoded_dims,))
  x=Dense(512)(inp)
  x=Activation('relu')(x)

  x=Dense(1500, )(x)
  x=Activation('relu')(x)
  x=Reshape(( 15, 25, 4))(x)
  
  x = Conv2DTranspose(4,(3,3), padding='same')(x)
  x=Activation('relu')(x)
  x = Conv2DTranspose(filters,(3,3), padding='same')(x)
  x=Activation('relu')(x)

  x = UpSampling2D((2,2))(x)
  x = Conv2DTranspose(filters,(3,3), padding='same')(x)
  x=Activation('relu')(x)
  x = Conv2DTranspose(filters,(3,3), padding='same')(x)
  x=Activation('relu')(x)

  x = UpSampling2D((2,2))(x)
  x = Conv2DTranspose(filters,(3,3), padding='same')(x)
  x=Activation('relu')(x)
  x = Conv2DTranspose(filters,(3,3), padding='same')(x)
  x=Activation('relu')(x)

  x = UpSampling2D((2,2))(x)
  x = Conv2DTranspose(filters,(3,3), padding='same')(x)
  x=Activation('relu')(x)
  x = Conv2DTranspose(filters,(3,3), padding='same')(x)
  x=Activation('relu')(x)
  x = Conv2DTranspose(1, (3, 3), padding='same')(x)
  x=Activation('sigmoid')(x)

  decoder=Model(inp,x)
  return decoder


In [0]:
encoder=None
decoder=None
autoencoder=None


In [0]:
# temp block
dataset=np.random.rand(14401,120,200,1)

In [0]:
if load_saved_model==False:
  inp=Input(shape=(dataset[0].shape))
  encoder=Encoder()
  decoder=Decoder()
  autoencoder = Model(inp, decoder(encoder(inp)))
  autoencoder.compile(optimizer='adam', loss='mse',metrics=['accuracy'])
#   autoencoder.compile(optimizer='rmsprop', loss='mse',metrics=['accuracy'])

  # save model architecture to json
  with open('{}/autoencoder.json'.format(path), 'w') as f:
    f.write(autoencoder.to_json())
  with open('{}/encoder.json'.format(path), 'w') as f:
    f.write(encoder.to_json())
  with open('{}/decoder.json'.format(path), 'w') as f:
    f.write(decoder.to_json())

#   encoder.summary()          
#   decoder.summary()
  autoencoder.summary()

  # # visualize model
  from IPython.display import SVG
#   from keras.utils.vis_utils import model_to_dot
  from keras.utils import plot_model

#   SVG(model_to_dot(autoencoder,show_shapes=True,show_layer_names=True,rankdir="TD").create(prog='dot', format='svg'))
  plot_model(encoder,show_shapes=True, to_file=path +'/encoder.png')
  plot_model(decoder,show_shapes=True, to_file=path +'/decoder.png')
  plot_model(autoencoder,show_shapes=True, to_file=path +'/autoencoder.png')

In [0]:
from keras.models import model_from_json
import json
encoder=None
decoder=None
autoencoder=None
if load_saved_model==True:
  encoder=load_model(path+'/encoder_weight.h5')
  decoder=load_model(path+'/decoder_weight.h5')
  autoencoder=load_model(path+'/../../ai_dance_aec_128_dims/autoencoder_weight.h5')
  encoder.summary()
  decoder.summary()
  autoencoder.summary()

  

> ## Callback

In [0]:
# suffix='autoencoder 128 dims'
suffix=''

In [0]:
# used to log training stats 
from keras.callbacks import Callback

plot_after=1
show_after=100
save_after=1
class MyLogger(Callback):
    def on_train_begin(self, logs={}):
      self.i = 0
      self.x = []
      self.losses = []
      self.val_losses = []
      self.acc = []
      self.val_acc = []
      self.logs = []
      self.max_acc=-9999
      self.max_val_acc=-9999
      self.min_loss=9999
      self.min_val_loss=9999
    
    def on_epoch_end(self, epoch, logs=None):
      self.update_value(logs)
      self.plot_graph(epoch)
      self.save_model(logs,epoch)
      self.write_log(logs,epoch)  

      
    def write_log(self,logs,epoch):
      with open('{}/log.txt'.format(path), 'a+') as f:
          f.write('\n#{} : {}'.format(epoch, logs))
          
    def update_value(self,logs):
      self.logs.append(logs)
      self.x.append(self.i)
      self.losses.append(logs.get('loss'))
      self.val_losses.append(logs.get('val_loss'))
      self.acc.append(logs.get('acc'))
      self.val_acc.append(logs.get('val_acc'))
      self.i += 1
    
      self.min_loss = min(logs.get('loss'),self.min_loss)
      self.min_val_loss = min(logs.get('val_loss') ,self.min_val_loss)
      self.max_acc = max(logs.get('acc'),self.max_acc)
      self.max_val_acc = max(logs.get('val_acc') ,self.max_val_acc)

    def plot_graph(self,epoch):
#     loss and accuracy plotting
      if epoch%plot_after==0:
        # LOSS         
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.savefig("{}/loss/graph_loss{}.png".format(path,suffix))
#         plt.show();
        plt.close()
        # ACC
        plt.plot(self.x, self.acc, label="acc")
        plt.plot(self.x, self.val_acc, label="val_acc")
        plt.legend()
        plt.savefig("{}/acc/graph_acc{}.png".format(path,suffix))
#         plt.show();
        plt.close()
  
#     Output saving
      dense=encoder.predict_on_batch(testd)
      print(dense[0])
      plot_dense(dense,epoch)
      pre=decoder.predict_on_batch(dense)
      fig = plt.figure(figsize=fig_size)
      for i in range(0,n):
        a1 = fig.add_subplot(n/pr,n/pr, i+1)
        p=a1.imshow(np.reshape(pre[i],(image_height,image_width),),cmap='gray')
        a1.get_xaxis().set_visible(False)
        a1.get_yaxis().set_visible(False)
#       if epoch%show_after==0:
#         print("*"*20,'RECREATION ',epoch,'*'*20)
#         plt.show()
#         print("*"*50)        
      plt.close()
      fig.savefig("{}/out/{}.png".format(path,epoch))
    
    
    
    def save_model(self,logs,epoch):
      if epoch%save_after==0:
        autoencoder.save("{}/autoencoder_weight_{}.h5".format(path,suffix))
        encoder.save("{}/encoder_weight_{}.h5".format(path,suffix))
        decoder.save("{}/decoder_weight_{}.h5".format(path,suffix))
      #saves best model:
      if logs.get('acc')==self.max_acc:
        autoencoder.save("{}/autoencoder_best_acc_{}.h5".format(path,suffix))
        encoder.save("{}/encoder_best_acc_{}.h5".format(path,suffix))
        decoder.save("{}/decoder_best_acc_{}.h5".format(path,suffix)) 
      if logs.get('val_acc')==self.max_val_acc:
        autoencoder.save("{}/autoencoder_best_val_acc_{}.h5".format(path,suffix,suffix))
        encoder.save("{}/encoder_best_val_acc_{}.h5".format(path,suffix))
        decoder.save("{}/decoder_best_val_acc_{}.h5".format(path,suffix))  
      if logs.get('loss')==self.min_loss:
        autoencoder.save("{}/autoencoder_least_loss_{}.h5".format(path,suffix))
        encoder.save("{}/encoder_least_loss_{}.h5".format(path,suffix))
        decoder.save("{}/decoder_least_loss_{}.h5".format(path,suffix)) 
      if logs.get('val_loss')==self.min_val_loss:
        autoencoder.save("{}/autoencoder_least_val_loss_{}.h5".format(path,suffix))
        encoder.save("{}/encoder_least_val_loss_{}.h5".format(path,suffix))
        decoder.save("{}/decoder_least_val_loss_{}.h5".format(path,suffix))
          
# correct one

> ## Training AUTOENCODER

In [0]:
reset(path)

# save 9 sample images for comparison
n=9
pr=3
fig_size=(10,10)
rand=[randint(0,3755) for i in range(n)]
testd=[dataset[rand[i]] for i in range(n) ]
testd = np.asarray(testd, dtype=np.float32)
start=0
fig = plt.figure(figsize=fig_size)
for i in range(n):
  a1 = fig.add_subplot(n/pr,n/pr, i+1)
  p=a1.imshow(np.reshape(testd[i],(image_height,image_width),),cmap='gray')
  a1.get_xaxis().set_visible(False)
  a1.get_yaxis().set_visible(False)
plt.close()
fig.savefig("{}/out/orignal_.png".format(path))
fig.savefig("{}/out/000_orignal_.png".format(path))

      
# callbacks   
from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau,CSVLogger,LambdaCallback
cbk=MyLogger()
bva= ModelCheckpoint(path+'/aec_best_val_acc_'+suffix+'.h5', monitor='val_acc', mode='max',save_best_only=True, save_weights_only=False)
bvl= ModelCheckpoint(path+'/aec_least_val_loss_'+suffix+'.h5', monitor='val_loss', mode='min',save_best_only=True, save_weights_only=False)
ba= ModelCheckpoint(path+'/aec_best_acc_'+suffix+'.h5', monitor='acc', mode='max',save_best_only=True, save_weights_only=False)
bl= ModelCheckpoint(path+'/aec_least_loss_model_'+suffix+'.h5', monitor='loss', mode='min',save_best_only=True, save_weights_only=False)
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.1,
                              patience=20, min_lr=0.0001,verbose=1)
csv_logger = CSVLogger(path+'/training.csv')
batch_size = 128
num_epoch = 1000
data=dataset

model_log = autoencoder.fit(data, data,
            batch_size=batch_size,
            epochs=num_epoch,
            verbose=1,
            validation_split=0.1,
            callbacks=[cbk,bva,bvl,ba,bl,reduce_lr,csv_logger],
            initial_epoch=0)



In [0]:
encoded_data=None
# encoded_dims=128
encoded_data=np.ndarray([])

for i in range(0,14401,500):
  x=encoder.predict_on_batch(dataset[i:i+500])
  encoded_data=np.append(encoded_data,x)
encoded_data=np.reshape(encoded_data[1:],(dataset.shape[0],encoded_dims))
np.save(dense_dataset_path,encoded_data)
encoded_data.shape

(14401, 128)

In [0]:
encoded_data[1000]

> ## RNN

In [0]:
load_dataset_from_npy = True 
load_saved_model=False  

path = F"ai_dance_models/ai_dance_128_dims/rnn" 
img_path = F"dataset/480_cleaned_14k_800x480/"
dense_dataset_path="dataset/dense_dataset.npy"
dataset_path="dataset/dataset.npy"
model_path=path+"/model_weight.h5"  
rnn_model_path=path+'/best_acc_model.h5'

In [0]:
# reset(path,False)
reset(path,True)

['ai_dance_models/ai_dance_128_dims/rnn/acc', 'ai_dance_models/ai_dance_128_dims/rnn/out', 'ai_dance_models/ai_dance_128_dims/rnn/loss', 'ai_dance_models/ai_dance_128_dims/rnn/dense']
do you really want to delete all folders i.e acc,loss,out (Y/N)?y
reset sucessfull :['ai_dance_models/ai_dance_128_dims/rnn/acc', 'ai_dance_models/ai_dance_128_dims/rnn/out', 'ai_dance_models/ai_dance_128_dims/rnn/loss', 'ai_dance_models/ai_dance_128_dims/rnn/dense']


In [0]:
import numpy as np
# data=encoded_data
data=np.load(dense_dataset_path)
train=data[:13000]
test=data[13000:-1]

In [0]:
def create_dataset(data,k):
    print('len',data.shape[0])
    dataX,dataY = [],[]
    for i in range(data.shape[0] - k):
        x = data[i:i+k]
        y = data[i+k]
        dataX.append(x)
        dataY.append(y)
    return np.array(dataX) , np.array(dataY)


In [0]:
look_back = 10
trainX ,trainY = create_dataset(train,look_back)
testX ,testY = create_dataset(test,look_back)
trainX.shape

len 13000
len 1400


(12990, 10, 128)

In [0]:
suffix=''
encoded_dims=128
def get_rnn():
  rnn = Sequential()
  rnn.add(LSTM(units = 512, return_sequences = True ,input_shape=trainX.shape[1:],))
  rnn.add(Dropout(0.2))
  rnn.add(LSTM(units = 256, return_sequences = True))
  rnn.add(Dropout(0.2))
  rnn.add(LSTM(units = 256, ))
  rnn.add(Dropout(0.2))
  rnn.add(Dense(128,activation='relu'))
  rnn.build(input_shape=trainX.shape[1:])
  return rnn
rnn=get_rnn()
rnn.summary()
with open('{}/rnn{}.json'.format(path,suffix), 'w') as f:
    f.write(rnn.to_json())
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
SVG(model_to_dot(rnn,show_shapes=True,show_layer_names=True,rankdir="TD").create(prog='dot', format='svg'))
from keras.utils import plot_model
plot_model(rnn,show_shapes=True, to_file=path +'/rnn{}.png'.format(suffix))



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 10, 512)           1312768   
_________________________________________________________________
dropout_7 (Dropout)          (None, 10, 512)           0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 10, 256)           787456    
_________________________________________________________________
dropout_8 (Dropout)          (None, 10, 256)           0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_9 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               32896     
Total para

In [0]:
# load saved model
rnn=None
from keras.models import model_from_json,load_model
import json
if load_saved_model==True:
  rnn=load_model(path+'/rnn_weight.h5')
  rnn.summary()
  

> ## Callback

In [0]:
# used to log training stats 
from keras.callbacks import Callback

plot_after=5
show_after=5
save_after=3
no_img_to_save=5
class MyLogger(Callback):
    def on_train_begin(self, logs={}):
      self.i = 0
      self.x = []
      self.losses = []
      self.val_losses = []
      self.acc = []
      self.val_acc = []
      self.logs = []
      
      ac=decoder.predict(testY[:no_img_to_save])
      num=0
      for im in ac:
        img=np.reshape(im,(120,200))
        plt.imshow(img,cmap='gray')
        plt.axis('off')
        plt.title(str('%04d orig'%(num)))
        plt.savefig(path+'/out/'+str('%04d orig'%(num))+'.jpg')
        num+=1


    #save model and plot result after each epoch 
    def on_epoch_end(self, epoch, logs=None):
      sample()
      epoch=epoch
      self.logs.append(logs)
      self.x.append(self.i)
      self.losses.append(logs.get('loss'))
      self.val_losses.append(logs.get('val_loss'))
      self.acc.append(logs.get('acc'))
      self.val_acc.append(logs.get('val_acc'))
      self.i += 1
      
      if epoch%plot_after==0:
        # LOSS         
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.savefig("{}/loss/graph_loss{}.png".format(path,suffix))
#         plt.show();
        plt.close()
#         ACC
        plt.plot(self.x, self.acc, label="acc")
        plt.plot(self.x, self.val_acc, label="val_acc")
        plt.legend()
        plt.savefig("{}/acc/graph_acc{}.png".format(path,suffix))
#         plt.show();
        plt.close()
   
                
      if epoch%save_after==0:
        rnn.save("{}/rnn_weight {}.h5".format(path,suffix))

      with open('{}/log.txt'.format(path), 'a+') as f:
          f.write('\n#{} : {}'.format(epoch, logs))
          
      if epoch%plot_after==0:
        testPredict = rnn.predict(testX[:no_img_to_save])
        res=decoder.predict(testPredict)    
        j=0
        for im in res:
          plt.close()
          img=np.reshape(im,(120,200))
          plt.imshow(img,cmap='gray')
          plt.axis('off')
          plt.title('epoch: '+str(epoch))
          plt.savefig(path+'/out/'+str('%04d'%(j))+'.jpg')
          plt.close()
          j+=1

          
# correct one

> ## Training Rnn

In [0]:
from keras.models import load_model
encoder=load_model(F"ai_dance_models/ai_dance_128_dims/aec/encoder_best_val_acc_.h5")
decoder=load_model(F"ai_dance_models/ai_dance_128_dims/aec/decoder_best_val_acc_.h5")

In [0]:
# reset(path,True)
from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau,CSVLogger
from keras import optimizers
cbk=MyLogger()
bva= ModelCheckpoint(path+'/best_val_acc_model'+suffix+'.h5', monitor='val_acc', mode='max',save_best_only=True, save_weights_only=False)
bvl= ModelCheckpoint(path+'/least_val_loss_model'+suffix+'.h5', monitor='val_loss', mode='min',save_best_only=True, save_weights_only=False)
ba= ModelCheckpoint(path+'/best_acc_model.'+suffix+'h5', monitor='acc', mode='max',save_best_only=True, save_weights_only=False)
bl= ModelCheckpoint(path+'/least_loss_model.'+suffix+'h5', monitor='loss', mode='min',save_best_only=True, save_weights_only=False)
csv_logger = CSVLogger(path+'/training2.csv')
if load_saved_model:
  rnn=rnn
else:
  rnn=get_rnn()
rnn.compile(loss = "mse",optimizer ='adam',metrics=['accuracy'],)

# rnn.summary()
batch_size = 128
num_epoch = 10000
# model training
model_log = rnn.fit(trainX, trainY,
            batch_size=batch_size,
            epochs=num_epoch,
            verbose=1,
            validation_data=(testX,testY),
            callbacks=[cbk,bva,bvl,ba,bl,csv_logger],
            initial_epoch=0)

In [0]:
from math import fabs
def sample():
#   rand=randint()
  r=rnn.predict(testX[:1])
  a=testY[:1]
  for i in range(1):
    print('*'*100)
    for j in range(10):
      print('%-12f%-12f%-12f'%(a[i][j],r[i][j],fabs(a[i][j]-r[i][j])))

> ## Result generation

In [0]:
load_dataset_from_npy = True 
load_saved_model=False  

path = F"ai_dance_models/ai_dance_128_dims/rnn" 
img_path = F"dataset/480_cleaned_14k_800x480/"
dense_dataset_path="dataset/dense_dataset.npy"
dataset_path="dataset/dataset.npy"
model_path=path+"/model_weight.h5"  
rnn_model_path=path+'/best_acc_model.h5'

In [0]:
from keras.models import load_model
encoder=load_model(F"ai_dance_models/ai_dance_128_dims/aec/encoder_best_val_acc_.h5")
decoder=load_model(F"ai_dance_models/ai_dance_128_dims/aec/decoder_best_val_acc_.h5")
rnn=load_model(F"ai_dance_models/ai_dance_128_dims/rnn/best_val_acc_model_.h5")

In [0]:
# !rm -r 'gdrive/My Drive/colab/ai_dance_models/testing/rnn_relu_activation/result'
# !mkdir 'gdrive/My Drive/colab/ai_dance_models/testing/rnn_relu_activationn/result'

In [0]:
rmtree(path+'/result')
os.mkdir(path+'/result')
# os.mkdir(path+'/video')

In [0]:
from shutil import rmtree
def reset_result_folder():
  res_folder=F'ai_dance_models/ai_dance_128_dims/rnn/result'
  rmtree(res_folder)
  os.mkdir(res_folder)
  print('RESETEDDD ',res_folder,)

In [0]:
reset_result_folder()

RESETEDDD  /content/gdrive/My Drive/colab/ai_dance_models/ai_dance_128_dims_10_lookup/rnn/result


In [0]:
def make_video(rate,fps,suffix):
  rate=rate #8
#   fps=rate
  fps=fps #24
  p='/'.join(path.split('/')[2:])
  img_dir=p+'/result/%6d.jpg'
  out_dir=p+'/video/result_rate_{}_fps_{}__{}.mp4'.format(rate,fps,suffix)
  print('Making video: ',out_dir)
  command="ffmpeg -r {} -i '{}' -c:v libx264 -vf fps={} -pix_fmt yuv420p '{}'".format(rate,img_dir,fps,out_dir,)
  print(command)
  os.system(command)
# print(os.system('ls' +path))

In [0]:
encoded_dims=128
look_back=10
def new_seed(old_seed,new_dense):
  
  x=np.asarray(new_dense)
  y=np.append(old_seed[0],x,axis=0)
  y=np.delete(y,0,axis=0)
  y=np.reshape(y,(1,look_back,encoded_dims))
  return y

In [0]:
import gc

def save_frame(im,frame_no):
  if frame_no%500==0:
    print("saved ",frame_no)
    print('garbage collector:',gc.collect())
  img=np.reshape(im,(120,200))
  plt.imshow(img,cmap='gray')
  plt.title(frame_no)
  plt.axis('off')
  plt.savefig(path+'/result/'+str('%06d'%(frame_no))+'.jpg')
  plt.close()


In [0]:
from random import randint
def generate_result(frames=500,interpolation=False,random_n=0):
  reset_result_folder()
#   random_n=randint(0,len(testX)-1)
  seed=np.random.rand(1,10,128)
#   seed=testX[random_n:random_n+1]
  frames=frames
  interpolation_frames=10
  video_rate=6
  video_fps=30
  
  if interpolation==False:
    for i in range(frames):
      den=rnn.predict(seed)
      frame=decoder.predict(den)
      save_frame(frame,i)
      seed=new_seed(seed,den)
    suffix="seed:{} _ No interpolation".format(random_n)
    make_video(video_rate,video_rate,suffix)
  else:
    new_den=np.zeros((1,128))
    old_den=np.zeros((1,128))
    frames=frames*(interpolation_frames)
    for i in range(0,frames,interpolation_frames):
#       if i%1000==0:
#         print(gc.get_count())
#         print(gc.collect())
      new_den=rnn.predict(seed)
      inc=(new_den-old_den)/interpolation_frames
      for interp_no in range(interpolation_frames):
        mid=old_den+inc*(interp_no+1)
        frame=decoder.predict(mid)
        save_frame(frame,i)
        i+=1
      seed=new_seed(seed,new_den)
      old_den=new_den
    suffix="seed:{} _interpolated".format(random_n)
#     video_rate=video_rate*interpolation_frames
    video_fps=120
    video_rate=40
    make_video(video_rate,video_fps,suffix)

      
      
      

  
  

In [0]:
seed=np.random.rand(1,10,128)


In [0]:
for i in range(1):
#   random_n=randint(0,len(testX)-1)
  random_n=1
  print('Using seed:: ',random_n)
  # print("********************************** Non interpolated **********************************")
  # generate_result(100,False,random_n)
  print("********************************** interpolated **********************************")
  generate_result(100,True,random_n)
  gc.collect()  

In [0]:
import os
zip_path='gdrive/My Drive/colab/ai_dance_models/ai_dance_128_dims_10_lookup'
zip_name=zip_path+'/../rnn and result.zip'
zip_command="zip -r '{}' '{}'".format(zip_name,zip_path)
print(zip_command)
os.system(zip_command)

In [0]:
def show_img(im,frame_no):
#   if frame_no%100==0:
#     print("saved ",frame_no)
  img=np.reshape(im,(120,200))
  plt.imshow(img,cmap='gray')
  plt.title(frame_no)
  plt.axis('off')
  plt.show()
#   plt.savefig(path+'/result/'+str('%04d'%(frame_no))+'.jpg')
#   plt.close()